## compile tflite_runtime with gpu support
this is a tutorial to compile a tflite_runtime python wheel with enabled gpu for debian aarch64 (raspberry / chromebook etc)

enables gpu in build script and sets cpu cores for compilation to 1 to prevent errors

In [ ]:
# ensure environment is up to date
!sudo apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [665 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,685 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [23.8 kB]
Get:13 http:/

In [ ]:
# get essential building software
!sudo apt install python3-pybind11 python3-pip wget curl cmake git sed -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sed is already the newest version (4.8-1ubuntu2).
wget is already the newest version (1.21.2-2ubuntu1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.15).
git is already the newest version (1:2.34.1-1ubuntu1.10).
Suggested packages:
  libeigen3-doc libmpfrc++-dev pybind11-doc python-numpy-doc python3-pytest
  python-setuptools-doc
The following NEW packages will be installed:
  libeigen3-dev pybind11-dev python3-numpy python3-pip python3-pybind11
  python3-setuptools python3-wheel
0 upgraded, 7 newly installed, 0 to remove and 86 not upgraded.
Need to get 6,502 kB of archives.
After this operation, 38.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pybind11-dev all 2.9.1-2 [146 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1

In [ ]:
#get missing pyconfig.h from debian server
import sys;
python_version = sys.version[:4]
if python_version[3] == ".":
  python_version = sys.version[:3]
path = "http://http.us.debian.org/debian/pool/main/p/python" + python_version + "/"
def switch(version):
    if version == "3.7.":
        return "libpython3.7-dev_3.7.3-2+deb10u3_arm64.deb"
    elif version == "3.9.":
        return "libpython3.9-dev_3.9.2-1_arm64.deb"
    elif version == "3.10":
        return "libpython3.10-dev_3.10.13-1_arm64.deb"
    elif version == "3.11":
        return "libpython3.11-dev_3.11.2-6_arm64.deb"
    elif version == "3.12":
        return "libpython3.12-dev_3.12.1-2_arm64.deb"

url = "" + path + switch(python_version)
filename= switch(python_version)
!wget $url
!mkdir tmp
!dpkg-deb -R $filename tmp
!cp --recursive tmp/usr/include/aarch64-linux-gnu /usr/include

--2024-01-30 15:10:24--  http://http.us.debian.org/debian/pool/main/p/python3.10/libpython3.10-dev_3.10.13-1_arm64.deb
Resolving http.us.debian.org (http.us.debian.org)... 208.80.154.139, 64.50.236.52, 64.50.233.100, ...
Connecting to http.us.debian.org (http.us.debian.org)|208.80.154.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3773028 (3.6M) [application/vnd.debian.binary-package]
Saving to: ‘libpython3.10-dev_3.10.13-1_arm64.deb’

libpython3.10-dev_3 100%[===================>]   3.60M  --.-KB/s    in 0.1s    

2024-01-30 15:10:24 (35.3 MB/s) - ‘libpython3.10-dev_3.10.13-1_arm64.deb’ saved [3773028/3773028]

rm: cannot remove 'tmp': No such file or directory


In [ ]:
# get tensorflow source
!git clone --single-branch --branch master https://github.com/tensorflow/tensorflow

Cloning into 'tensorflow'...
remote: Enumerating objects: 1666744, done.
remote: Counting objects: 100% (945/945), done.
remote: Compressing objects: 100% (433/433), done.
remote: Total 1666744 (delta 570), reused 859 (delta 507), pack-reused 1665799
Receiving objects: 100% (1666744/1666744), 934.79 MiB | 21.63 MiB/s, done.
Resolving deltas: 100% (1359590/1359590), done.
Updating files: 100% (31212/31212), done.


In [ ]:
# enable GPU in build script
!sed -i -e 's/-DCMAKE_CXX_FLAGS=/-DTFLITE_ENABLE_GPU=ON -DCMAKE_CXX_FLAGS=/g' tensorflow/tensorflow/lite/tools/pip_package/build_pip_package_with_cmake.sh

In [ ]:
# lower cpu cores to circumvent memmory errors
!sed -i -e 's/-j ${BUILD_NUM_JOBS}/-j2/g' tensorflow/tensorflow/lite/tools/pip_package/build_pip_package_with_cmake.sh

In [8]:
# build tflite_runtime
!PYTHON=python3 tensorflow/tensorflow/lite/tools/pip_package/build_pip_package_with_cmake.sh aarch64

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
-- Generating done (0.0s)
-- Build files have been written to: /content/tensorflow/tensorflow/lite/tools/pip_package/gen/tflite_pip/python3/cmake_build/pthreadpool-download
[ 11%] Creating directories for 'pthreadpool'
[ 22%] Performing download step (download, verify and extract) for 'pthreadpool'
-- Downloading...
   dst='/content/tensorflow/tensorflow/lite/tools/pip_package/gen/tflite_pip/python3/cmake_build/pthreadpool-download/pthreadpool-prefix/src/4fe0e1e183925bf8cfa6aae24237e724a96479b8.zip'
   timeout='none'
   inactivity timeout='none'
-- Using src='https://github.com/Maratyszcza/pthreadpool/archive/4fe0e1e183925bf8cfa6aae24237e724a96479b8.zip'
-- [download 7% complete]
-- [download 18% complete]
-- [download 34% complete]
-- [download 57% complete]
-- [download 83% complete]
-- [download 100% complete]
-- verifying file...
       file='/content/tensorflow/tensorflow/lite/tools/pip_package/gen/tflite_pip/pytho

In [10]:
# start download of python wheel to your workstation
#!pip3 install tensorflow/tensorflow/lite/tools/pip_package/gen/tflite_pip/python3/dist/tflite_runtime*.whl
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

import pathlib
path = pathlib.Path().resolve()
print(path)

filename = find('*.whl', path)
print(filename)
from google.colab import files
files.download(filename[0])

/content
['/content/tensorflow/tensorflow/lite/tools/pip_package/gen/tflite_pip/python3/dist/tflite_runtime-2.16.0-cp310-cp310-linux_aarch64.whl']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>